# HOG 알고리즘과 보행자 검출
---


그래디언트 방향 히스토그램을 의미  

히스토그램은 총 256개의 값이 나온다.  

영상 크기에 관계없이 256개의 특징을 구할 수 있는 특징  

엣지의 양을 보는 것이 아닌 엣지의 방향을 봄.  

서포트 벡터 머신을 사용하여 입력 영상에서 보행자 위치를 검출하는 방법  



하라이크와 검출방법은 동일함.  

- 기본적으로 64 x 128 크기의 영상에서 계산함
- 먼저 입력 영상으로부터 그래디언트를 계산 (소벨연산)
- 그래디언트는 방향 성분으로 계산하며, 방향성분은 0도부터 180도까지로 설정
- 전체영상을 8x8 픽셀로 나누어 가로 8개 세로 16개로 생성
- 각각의 셀로부터 그래디언트 방향 성분에 대한 히스토그램을 구함
    히스토그램의 장점으로 
- 방향 성분을 20도 단위로 구분하여 총 아홉개의 빈으로 구성된 방향 히스토그램이 만들어짐.
- 하나의 블록에는 네 개의 셀이 있고 각 셀에는 아홉개의 빈으로 히스토그램 저ㅇ보가 있음.
- 3780 개의 실수 값이 64X128 영상을 표현하는 HOG 특징 벡터 역할을 함


In [3]:
import sys
import cv2
import numpy as np
import random
import pytesseract
from IPython.core.display import HTML
import requests

from matplotlib import pyplot as plt
%matplotlib inline

def imshow(tit, image) :
    plt.title(tit)    
    if len(image.shape) == 3 :
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    else :
        plt.imshow(image, cmap="gray")
    plt.show()

- 두 특징 벡터를 구분하기 위해 SVM 알고리즘을 사용  
= SVM 알고리즘은 마진이 최대가 되는 초평면을 만듬

In [6]:
cap = cv2.VideoCapture('vision/vtest.avi')

hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

while True:
    ret, frame = cap.read()

    if not ret: break

    detected, _ = hog.detectMultiScale(frame)

    for (x, y, w, h) in detected:
        c = (random.randint(0, 255),
             random.randint(0, 255), 
             random.randint(0, 255))
        cv2.rectangle(frame, (x, y), (x + w, y + h), c, 3)

    cv2.imshow('frame', frame)
    if cv2.waitKey(10) == 27:  break

cv2.destroyAllWindows()


In [5]:
cap = cv2.VideoCapture('vision/vtest.avi')

hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

fgbg = cv2.createBackgroundSubtractorMOG2(varThreshold=100)


while True:
    ret, frame = cap.read()
    if not ret: break
        
    fgmask = fgbg.apply(frame)        
    detected, _ = hog.detectMultiScale(frame)

    for (x, y, w, h) in detected:
        c = (random.randint(0, 255),
             random.randint(0, 255), 
             random.randint(0, 255))
        cv2.rectangle(frame, (x, y), (x + w, y + h), c, 3)        
    cv2.imshow('mask', fgmask)
    cv2.imshow('frame', frame)
        
    if cv2.waitKey(-1) == 27:  break

cv2.destroyAllWindows()

# 지역 특징점 검출과 매칭
---